In [ ]:
! pip install streamlit -q

the cell python code into an app.py file

In [ ]:
!unzip "dataset_a.zip" -d ""
!unzip "dataset_bc.zip" -d ""

Archive:  dataset_a.zip
  inflating: dataset_a.csv           
Archive:  dataset_bc.zip
  inflating: dataset_bc.csv          


In [ ]:
! pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', '{:.0f}'.format)
import pydeck as pdk
import plotly.express as px

st.set_option('deprecation.showPyplotGlobalUse', False)


df_1 = pd.read_csv('dataset_a.csv')
df_1.head()

df_1['datetime'] = pd.to_datetime(df_1['datetime'],format='%Y%m%d%H%M%S')
df_2 = pd.read_csv('dataset_bc.csv')
df_2['datetime'] = pd.to_datetime(df_2['datetime'],format='%Y%m%d%H%M%S')

df = pd.concat([df_1,df_2])


import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
from xgboost import XGBClassifier
import xgboost as xgb
from xgboost import plot_importance
#import joblib
from sklearn import metrics
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split, BaseCrossValidator
import time,datetime
import sqlite3
import pandas.io.sql as psql
import csv,os,imp
from sklearn.utils import class_weight
from sklearn import manifold, datasets
from sklearn.preprocessing import normalize,Normalizer,StandardScaler,MinMaxScaler,scale 
train = df[(df['traintest']==0) & (df['abc']=='c')]  #a,c
test = df[(df['traintest']==1) & (df['abc']=='a')] #a,c
date_1 = train['datetime'] 
date_2 = test['datetime']
y_train = train['labelnew'].values  

train = train.drop(['labelnew','abc','traintest','datetime','node'
                    ],axis=1)

X_train =  train.values  

y_test = test['labelnew'].values

test = test.drop(['labelnew','abc','traintest','datetime','node'
                    ],axis=1)
X_test =  test.values  

print("----------------------- Xgboost start ---------------------")
start_time = time.time()

clf = XGBClassifier(
    silent=0,  # 
    nthread = -1, # 
    tree_method= 'exact', #'approx', #'hist', #'approx', #approx,exact,hist
    #max_bin=12, #240, #default=256
    booster='gbtree',#dark  gbtree  
    #booster='dark',#dark
    #sample_type='weighted', #uniform
    #normalize_type='forest', #tree
    #rate_drop=0.9,   
    n_estimators=80,  # 
    max_depth=4, #
    learning_rate=0.2, #
    #min_child_weight = 0.9, default=1
    max_delta_step=1, #
    #scale_pos_weight = 130, # 1778034/986# 
    #base_score=0.90, #
    #reg_lambda=10, #default=1
    #gamma=0,
    subsample=0.85, #
    #gamma = 0.1,# 0.1~0.2
    #subsample=1, # 
    colsample_bytree=0.8, # 
    #reg_lambda=1, #
    #reg_alpha=0, # L1
    #max_delta_step=2,#1  # 
    #scale_pos_weight = 0.0017499，#sum(negative instances)/sum(positive instances)
    #booster='gbtree',#dark
    objective ='binary:logistic',#'binary:logistic', #'multi:softprob', #'multi:softmax' 
    #base_score=0.66,worse
    #bjective ='multi:softprob',#'binary:logistic', #'multi:softprob', #'multi:softmax' 
    #um_class = 2,  # multisoftmax
    #seed = 1440,  # 
    #missing=None,
    #eval_metric ='auc',#error
    )
import joblib
print('make model')
clf.fit(X_train, y_train)
joblib.dump(clf, 'Xgboost.pkl')

print("Xgboost training cost time : ",time.time()-start_time)

print('# -----------------6: Xgboost: train scoring ---------------')
#load model
rfcbuild = joblib.load('Xgboost.pkl')
TrainResult = rfcbuild.predict(X_train)

colNum = TrainResult.shape[0]
TrainResult = TrainResult.reshape(colNum,1)

print("confusion_matrix:")
print(confusion_matrix(y_train,TrainResult))
print('---------')
print(classification_report(y_train,TrainResult))
roc_auc1 = roc_auc_score(y_train,TrainResult)
print("Area under the ROC curve : %f" % roc_auc1)

print("------------------ Xgboost: test scoring ---------------")

TestResult = rfcbuild.predict(X_test)
colNum = TestResult.shape[0]
TestResult = TestResult.reshape(colNum,1)

print("confusion_matrix:")
print(confusion_matrix(y_test,TestResult))
print('---------')
print(classification_report(y_test,TestResult))
roc_auc1 = roc_auc_score(y_test,TestResult)
print("Area under the ROC curve : %f" % roc_auc1)

# Feature Importance
plt.figure(figsize=(10, 5))
fea = rfcbuild.feature_importances_
maxrange=np.max(fea)-np.min(fea)

fea_normed = 100 * fea/np.max(fea)
#fea_normed = fea/maxrange
#fea_normed = (fea-np.min(fea))/maxrange
#print('Importance score: ',type(fea_normed),fea_normed.shape,fea_normed)
np.set_printoptions(suppress=True)
print('Importance score: ')
print(np.round(fea_normed,2))

plt.figure(figsize=(10, 5))
plt.bar(range(len(rfcbuild.feature_importances_)), fea_normed)
plt.title("Xgboost: Feature Importance")
plt.xlabel("Features")
plt.ylabel("Normalized Importance Score")
plt.show()  

print(' ---------  Output Test (predict) results to csv  -----')
print('----------- Xgboost train and testsuccessful completed-----------------')

x_train = pd.DataFrame(X_train,columns = train.columns)
x_test = pd.DataFrame(X_test,columns = test.columns)
importance = fea_normed.tolist()
index = [i for i, e in enumerate(importance) if e >0]

x_train_new = x_train.iloc[:,index]
x_train_new = pd.concat([date_1.reset_index(drop=True),x_train_new,pd.Series(y_train)],axis=1)


x_test_new = x_test.iloc[:,index]
x_test_new = pd.concat([date_2.reset_index(drop=True),x_test_new,pd.Series(y_test)],axis=1)


data = pd.concat([x_train_new,x_test_new])


data = data.rename(columns = {0:'label'})

data.isna().sum()
data = data.fillna(method='bfill')


df_sorted=data.sort_values(by='datetime')
df_sorted = df_sorted.set_index(pd.DatetimeIndex(df_sorted['datetime']))
df_sorted.drop(columns ='datetime',inplace = True)

#120
def plot_sorted():
  for col in df_sorted.columns[:-1]:   
   
    plt.figure(figsize=(15,8))
    plt.plot(df_sorted.index,df_sorted[col])
    plt.xlabel('Date')
    plt.ylabel(col)
    plt.title(f'Date Vs {col}')
    plt.show()


df_sorted = df_sorted.iloc[:,2:]

def plot_box_plot():
  plt.figure(figsize=[15,8])
  sns.boxplot(data=df_sorted)
  plt.xticks(rotation=90)

x=plot_box_plot()  


def time_series_visualization():
  plt.style.use('fivethirtyeight')
  df_sorted.plot(subplots=True,
        layout=(6, 5),
        figsize=(22,22),
        fontsize=10, 
        linewidth=2,
        sharex=False,
        title='Visualization of the original Time Series')
  plt.show()



#df_transform
df_transform = df_sorted.loc[:,:'software-oper_cisco-platform-software_control-processes_control-process_per-core-stats_per-core-stat_idle.1'].copy().pct_change(1)

# fill the 1st row with NA data
df_transform.fillna(method='bfill', inplace=True)
df_transform = df_transform.replace(np.inf,np.nan)
# fill the 1st row with NA data
df_transform.fillna(method='bfill', inplace=True)




# Plot the transformed time series
def plot_transformed_feature():
    plt.style.use('fivethirtyeight')
    df_transform.plot(subplots=True,
                      layout=(6, 5),
                      figsize=(24,24),
                      fontsize=10, 
                      linewidth=2, 
                      title='Visualization of the transformed Features')
    plt.show()


# Split train and test data
train_features = df_transform.loc['2021-01-28 17:44:40':'2021-02-08 17:44:40']
train_labels = df_sorted.loc['2021-01-28 17:44:40':'2021-02-08 17:44:40', 'label']

test_features = df_transform.loc['2021-02-08 17:44:40':'2021-02-10 13:37:40']
test_labels = df_sorted.loc['2021-02-08 17:44:40':'2021-02-10 13:37:40', 'label']

# I want to use a T-days window of input data for predicting target_class
# It means I need to prepend (T-1) last train records to the 1st test window
T = 45  # my choice of the timesteps window

prepend_features = train_features.iloc[-(T-1):]
test_features = pd.concat([prepend_features, test_features], axis=0)

train_features.shape, train_labels.shape, test_features.shape, test_labels.shape




# Rescale the features
from sklearn.preprocessing import StandardScaler  # MinMaxScaler

scaler = StandardScaler()  # MinMaxScaler(feature_range=(-1,1))
scaled_train_features = pd.DataFrame(scaler.fit_transform(train_features.values),
                                     index=train_features.index,
                                     columns=train_features.columns)
# The Scaler is fit on the training set and then applied to the test set
scaled_test_features = pd.DataFrame(scaler.transform(test_features.values),
                                    index=test_features.index,
                                    columns=test_features.columns)

scaled_train_features.shape, scaled_test_features.shape







# Plot the rescaled_train_features
def rescaled_train_features():
    plt.style.use('fivethirtyeight')
    scaled_train_features.plot(subplots=True,
                              layout=(6, 5),
                              figsize=(24,24),
                              fontsize=10, 
                              linewidth=2, 
                              title='Visualization of the scaled Train Features')
    plt.show()











  


st.title("EXplortory data analysis for LSTM ")
menu = ["Home","EDA","About"]
choice = st.sidebar.selectbox('Menu',menu)
if choice == 'Home':
    st.title("Choose From Menu ^_^ ")
    st.title("<Exploratory Data Analysis for LSTM > Or <Home>")
elif choice == 'EDA':
    st.subheader("Box PLot Visualization ")
    box=plot_box_plot()  
    st.pyplot(box)	

    st.subheader("Time Series Visualization From The Original Data ")
    p_timeseries=time_series_visualization()
    st.pyplot(p_timeseries)

    st.subheader("Time Series Visualization From The Transformed Feature ")
    plot_transformed=plot_transformed_feature()
    st.pyplot(plot_transformed)

    st.subheader("Time Series Visualization From The Rescaled Train Feature ")
    rescaled_train=rescaled_train_features()
    st.pyplot(rescaled_train)




elif choice == 'About':
    st.subheader("About page ")    
else:
    st.subheader("nothing page ") 






Overwriting app.py


In [ ]:
! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2FOkOpGt9VvDInSVYVgDmtyw7Vx_6zgpm4qtf1EMzgFUxqoVa") #ngrok.com

In [ ]:
!nohup streamlit run app.py --server.port 80 &

nohup: appending output to 'nohup.out'


In [ ]:
url = ngrok.connect(port = '80')
print(url)

NgrokTunnel: "http://989c-35-227-108-163.ngrok.io" -> "http://localhost:80"


In [ ]:
from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
tunnels

# try:
#     # Block until CTRL-C or some other terminating event
#     ngrok_process.proc.wait()
# except KeyboardInterrupt:
#     print(" Shutting down server.")

#     ngrok.kill()

[<NgrokTunnel: "https://989c-35-227-108-163.ngrok.io" -> "http://localhost:80">,
 <NgrokTunnel: "http://989c-35-227-108-163.ngrok.io" -> "http://localhost:80">]

Run streamlit using `localtunnel` 

npm installation would take some time for the first time

In [ ]:
ngrok.kill()